# Random Forest
Train on 6-class cancer classification.  
Input is the nucleus+RBC rollup, training set only.  
Train on the full training set (no validation).  
Rank feature importance.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2022-06-09 10:48:24.607706
Python 3.8.10
sklearn 1.0.2


[]

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models
from CellProfiler_Util import CP_Util
from RandomForestUtil import RF_Util

In [3]:
BASE_DIR='/home/jrm/Adjeroh/Naved/CP_80K/'  # append Output0/ etc
CLASSES=range(0,6)  # use all 6 classes
#CLASSES=[5]  # JUST FOR TESTING
MODELS_DIR='/home/jrm/Adjeroh/Naved/CP_80K/models/RandomForest.17'

In [4]:
print(datetime.datetime.now())
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_all_classes():
    X = None
    y = None
    for i in CLASSES:
        print(datetime.datetime.now())
        print("Process class",i)
        FULL_PATH=BASE_DIR+'Output'+str(i)+'/'
        cp = CP_Util(FULL_PATH)
        cp.train_test_split()
        Xnuc = cp.get_nucleus_rollup()
        set_prefix('Nuc_')
        Xnuc.rename(get_prefix,axis='columns',inplace=True)
        Xrbc = cp.get_RBC_rollup()
        set_prefix('Rbc_')
        Xrbc.rename(get_prefix,axis='columns',inplace=True)
        Xi = Xnuc.join(Xrbc,how='outer')
        size = len(Xi)
        yi = np.ones(size) * i   # e.g. class 3
        if X is None:
            X = Xi
            y = yi
        else:
            X = pd.concat( (X,Xi) )
            y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
Xtrain,ytrain=load_all_classes()
Xtrain

2022-06-09 10:48:25.942721
2022-06-09 10:48:25.942981
Process class 0
Num tumors in test/train sets: 23 94
Num patches in test/train sets: 8803 36163
2022-06-09 10:49:05.943921
Process class 1
Num tumors in test/train sets: 7 26
Num patches in test/train sets: 2734 10168
2022-06-09 10:49:18.342411
Process class 2
Num tumors in test/train sets: 7 30
Num patches in test/train sets: 2769 11181
2022-06-09 10:49:32.746401
Process class 3
Num tumors in test/train sets: 3 14
Num patches in test/train sets: 1091 5273
2022-06-09 10:49:45.279427
Process class 4
Num tumors in test/train sets: 2 6
Num patches in test/train sets: 813 2384
2022-06-09 10:49:54.501098
Process class 5
Num tumors in test/train sets: 1 3
Num patches in test/train sets: 396 1195


,Nuc_ObjectNumber_count,Nuc_ObjectNumber_mean,Nuc_ObjectNumber_std,Nuc_ObjectNumber_min,Nuc_ObjectNumber_25%,Nuc_ObjectNumber_50%,Nuc_ObjectNumber_75%,Nuc_ObjectNumber_max,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,...,Rbc_Number_Object_Number_75%,Rbc_Number_Object_Number_max,Rbc_Parent_ShrinkRBC_count,Rbc_Parent_ShrinkRBC_mean,Rbc_Parent_ShrinkRBC_std,Rbc_Parent_ShrinkRBC_min,Rbc_Parent_ShrinkRBC_25%,Rbc_Parent_ShrinkRBC_50%,Rbc_Parent_ShrinkRBC_75%,Rbc_Parent_ShrinkRBC_max
PatchNumber,,,,,,,,,,,,,,,,,,,,,
397,7.0,4.0,2.160247,1.0,2.50,4.0,5.50,7.0,7.0,216.000000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
398,1.0,1.0,0.000000,1.0,1.00,1.0,1.00,1.0,1.0,200.000000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
399,1.0,1.0,0.000000,1.0,1.00,1.0,1.00,1.0,1.0,181.000000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
400,4.0,2.5,1.290994,1.0,1.75,2.5,3.25,4.0,4.0,239.250000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
401,4.0,2.5,1.290994,1.0,1.75,2.5,3.25,4.0,4.0,237.500000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,7.0,4.0,2.160247,1.0,2.50,4.0,5.50,7.0,7.0,368.285714,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1588,17.0,9.0,5.049752,1.0,5.00,9.0,13.00,17.0,17.0,519.352941,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1589,19.0,10.0,5.627314,1.0,5.50,10.0,14.50,19.0,19.0,388.315789,...,1.75,2.0,2.0,1.5,0.707107,1.0,1.25,1.5,1.75,2.0


In [6]:
print(datetime.datetime.now())
print("Shuffle...")
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

2022-06-09 10:50:19.368698
Shuffle...


In [7]:
print(datetime.datetime.now())
print('Train on full training set...')
rf3 = RF_Util()
rf3.set_train(Xtrain,ytrain)
rf3.fit()
print(datetime.datetime.now())
print('...and rank the features by importance.')
top = rf3.important_features()

2022-06-09 10:50:21.553224
Train on full training set...
2022-06-09 10:58:36.464579
...and rank the features by importance.


In [8]:
pd.set_option('display.max_rows', None)
top.loc[:100]

,0,1
0,0.007890,Nuc_AreaShape_MedianRadius_75%
1,0.006071,Nuc_Texture_InfoMeas1_Hematoxylin_4_00_256_75%
2,0.005058,Nuc_Texture_InfoMeas1_Hematoxylin_7_00_256_50%
3,0.004978,Nuc_Texture_InfoMeas1_Hematoxylin_5_03_256_50%
4,0.004977,Nuc_Texture_InfoMeas1_Hematoxylin_5_02_256_mean
5,0.004694,Nuc_Texture_InfoMeas1_Hematoxylin_5_01_256_mean
6,0.004652,Nuc_Texture_InfoMeas1_Hematoxylin_4_03_256_50%
7,0.004597,Nuc_AreaShape_MedianRadius_mean
8,0.004479,Nuc_AreaShape_EquivalentDiameter_25%
9,0.004448,Nuc_Texture_InfoMeas1_Hematoxylin_7_03_256_mean


In [9]:
print(datetime.datetime.now())
joblib.dump(rf3.get_model(),MODELS_DIR)

2022-06-09 10:58:36.654790


['/home/jrm/Adjeroh/Naved/CP_80K/models/RandomForest.17']